In [61]:
# import required module
import os
import CFG as cfg
import re
from uuid import uuid4
import csv
import json
import copy

# python retdec-decompiler.py yourbinary.exe --stop-after bin2llvmir

In [62]:
def print_nodes(nodes):
    #print node data in dictionary format
    print(len(nodes))
    for ID in nodes.keys():
        print("node data")
        print("label: ", nodes[ID].label)
        print("is_entry: ", nodes[ID].is_entry)
        print("is_exit: ", nodes[ID].is_exit)
        for ir in nodes[ID].instructions:
            print(ir)
        # print("=> edges in \n")
        # for edge in nodes[ID].edges_in:
        #         print("from  "+ edge.source_id + " \n")
        #         print("to  "+ edge.target_id + " \n")
        # print("=> edges out \n")
        # for edge in nodes[ID].edges_out:
        #     print("from  "+ edge.source_id + " \n")
        #     print("to  "+ edge.target_id + " \n")
        print("-----------------")

In [63]:
'''
check number of functions thier end and start
number of nodes totally
number of edges (+ calls)
then check src and target of each edge
'''

'\ncheck number of functions thier end and start\nnumber of nodes totally\nnumber of edges (+ calls)\nthen check src and target of each edge\n'

In [64]:
def extract_functions(filename):
    '''
    this function take the IR file name passes through the its lines
    and returns a dictionary of all functions in the file and list of their names
    '''
    functions_lines={} # store each function lines with key as index 0,1,2 .. representing the position of the function in the file
    function_names=[]
    lines=[] # temporary list of lines for each function
    start = False
    end=False
    index =0
    #loop on file lines
    with open(filename, 'r') as f:
        for line in f:
                # start storing lines withe the firs fuction
                if not start:
                    start_found=re.search("^define.+", line)
                if start_found:
                    start=True
                if start:
                    lines.append(line)
                
                end_found=re.search("^}$", line, re.MULTILINE)
                if end_found:
                    functions_lines[index]=lines
                    lines=[]
                    index+=1
                    start=False
                    if start_found:
#TODO exclude bracket from .+ 3shan da @mingw_set_invalid_parameter_handler(void (i16*, i16*, i16*, i32, i64) bytl3 el name kda @mingw_set_invalid_parameter_handler(void
                        temp=re.search("@.+(?=\()", start_found[0])
                    if temp:
                        function_names.append(temp[0])
                    else:
                        print("not quak!!!!!!!!")
                        print(start_found[0])
    f.close()
    return functions_lines, function_names


In [65]:
def construct_nodes(lines,function_id, function_name):
    '''
    this function stores the nodes' data for each function
    input: lines of the function, id and name of the function
    return a dictionary of nodes contains all basic blocks of the functions nodes' objects
    with all its data and a list of tuples contain all calls to other functions in this function
    '''
    nodes={}
    # data needed for each function and each node inside the function
    node_start=False
    IRs=[]
    # repeated names functions?
    calls=[] # list of tuples (function_name, source_node_id)
    label=""
    is_entry=False
    i=0
    function_entry=() # tuple to be stored in dict to access the entry node of each function easily by its name
    if function_name == "@_gnu_exception_handler":
        print("yooooh switch")
        for line in lines:
            print(line)
            print("-----------------")
        print("yooooh switch")
    for line in lines:
        #1---------- start of a node (basic block)
        if not node_start:
           
            #node_start_found=re.search("(?<=^dec_label_pc_)[0-9a-z]+(?=:)", line)
            node_start_found=re.search("(?<=^dec_label_pc_).+(?=:)", line)
           
        if node_start_found:
            node_start=True
            label = node_start_found[0]
            if label=="140002000":
                print("ML")
                print(node_start_found)
            
           
            node_start_found=None #3shan mad5olsh a-search kol mara 3la awl line 
            # print(label)
            # print(node_id)
            if i==0:
                is_entry=True #this the first basic block in the function
                function_entry=(function_name,label)
                i+=1
            continue #TODO CHECK ANA SHLTHA 3SHAN MYSGLSH AWL IR (DEC LABEL..)
        #2-------- store IRs
        if node_start:
            #TODO does we need to store the label instruction?
            IRs.append(line)
            #3-------- store calls
            call_found=re.search("(?<=call ).+", line)
            if call_found:
                temp=re.search("@\w+(?=\()", call_found[0])
                if temp:
                    # called function name , source node label, calling function
                    calls.append((temp[0],label, function_name))
              
        #4------------- end of a node (basic block)
        node_end_found=re.search("( ret| br| switch | unreachable| resume).+", line)
        if label=="140001760":
            print("ML552")
            print(line)
            print(node_end_found)
        # when reaching end of the basic block store the node data
        if node_end_found:
            
            #----create object for the new node and add it to the dictionary
            nodes[label]=cfg.node("",label, is_entry,False,[], [],IRs)
            nodes[label].function_id=function_id
            nodes[label].function_name=function_name
            #--- reset variables
            IRs=[]
            node_start=False
            node_end=False
            label=""
            is_entry=False
            node_end_found=None
    
    return nodes ,calls,function_entry
        
        
                

In [66]:
def construct_edges (nodes,functions_entry,calls):
    '''
    this function takes the nodes dictionary and construct the edges between them
    take functions_entry to get entry node of the calling function
    take calls to get all calling function for any function to connect ret instructions
    return list of edge objects contains labels of source and target nodes (basic blocks)
    '''
    edges=[]
    #TODO need to handle other terminators
    for ID in nodes.keys():
        # the last instruction in the basic block is the branch
        # to guarantee that the node is not empty (sometimes store)
        if len(nodes[ID].instructions) ==0:
            continue
        
        ir = nodes[ID].instructions[-1]
        if ID == "1400013c8":
            print("luca")
            print(ir)
       
        src =nodes[ID].label

        branch_found="br " or "switch " in ir #TODO need rejex?
        ret_found ="ret " in ir #TODO need rejex? 
        if branch_found:
            #TODO check rule of writing labels
            
            targets=re.findall("(?<=%dec_label_pc_)[0-9a-z\.]+(?=\W)", ir)
           
            for target in targets:
                new_edge= cfg.edge ( src, target)
                edges.append(new_edge)
                nodes[ID].edges_out.append(new_edge) 
        #TODO Check: daymn bnwsl b el entry node w da msh s7?? l2n msh daymn el 3aml call f el entry node f el function
        # TODO ret el main msh bttwsl m7dsh by3mlha call  
        if ret_found:
            for call in calls:
                # in all calls find the call where the called function the function containing this ret instruction
                # connect it with all calling functions
                # print(call[0])
                # print(nodes[ID].function_name)
                if call[0] == nodes[ID].function_name:
                    # print("????")
                    #get entry of calling function
                    # print(call[2])
                    #target = functions_entry[call[2]]
                    target = call[1]
                    new_edge= cfg.edge (src, target)
                    edges.append(new_edge)
                    nodes[ID].edges_out.append(new_edge)
                    #print("created edge")
    return edges

            



In [67]:
def connect_functions (nodes , calls,functions_entry):
    '''
    this function takes the nodes dictionary and the list of calls to other functions
    and connect the nodes of the current function to the nodes of the called functions
    return the extra list of edges and the  updated nodes dictionary 
    '''
    # print("function names")
    # print(functions_entry)
    call_edges=[]
    for call in calls:
        # (function_name, source_node_id)
        src=call[1]
        # the target node   is the entry node of the called function
        if call[0] in functions_entry.keys():
            target=functions_entry[call[0]]
            new_edge= cfg.edge (src, target)
            call_edges.append(new_edge)
            nodes[call[1]].edges_out.append(new_edge)
            #TODO does we need to add edges in of the targets? i don't have their IDs
    
    return call_edges, nodes
        

In [68]:
def create_dataFrame(edges,file_name):
      
    sources=[]
    targets=[]
    with open('edges/edges_'+file_name+'.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        for edge in edges:
            writer.writerow( [edge.source_id, edge.target_id])
            # sources.append(edge.source)
            # targets.append(edge.target)
  
    # assign data of lists.  
    # data = {'source': sources, 'target': targets}  
    
    # # Create DataFrame  
    # df = pd.DataFrame(data)  
    
    # # Print the output.  
    # print(df)  

In [69]:
def create_json(nodes, file_name):
    nodes_IRs={}
    for ID in nodes.keys():
        nodes_IRs[ID]=nodes[ID].instructions 
    jsonString = json.dumps(nodes_IRs)
    jsonFile = open('jsons/'+file_name+"_json.json", "w")
    jsonFile.write(jsonString)
    jsonFile.close()

In [70]:
def handle_switch(function_lines):
    
    for i in function_lines.keys():
        switch_found=False
        new_ir=""
        temp_ir=copy.deepcopy(function_lines[i]) # copy the list to iterate over it as we wil remove elements from the original list
        #store index of the switch instruction to store the whole switch instruction in one line in its correct index
        switch_index=-1 
        if i == 2 or i==92 or i==54:
            print("tooooll1")
            print(i)
            print(len(temp_ir))
            print(len(function_lines[i]))
        for ele in enumerate(temp_ir):
            if "switch " in ele[1] and not switch_found:
                print("switch found")
                print(ele[0])
                switch_found=True
                function_lines[i].pop(ele[0])
                switch_index=ele[0]
                new_ir=ele[1]
                continue
            if switch_found:
                function_lines[i].pop(ele[0])
                new_ir+=ele[1]
                # print("switch found2")
                # print(ele[1])
                # print(new_ir)
                # print(new_ir)
                #end of the switch instruction
                if "]" in ele[1]:
                    switch_found=False
                    # print("switch end found")
                    # print(ele[1])
                    # print(new_ir)
                    # print("RFFFFFFFFFFFFFFFff")
                    # print(new_ir)
                    function_lines[i].insert(switch_index,new_ir)
                    new_ir=""
                    switch_index=-1
                    
        if i == 2 or i==92 or i==54:
            print("tooooll2")
            print(i)
            print(len(temp_ir))
            print(len(function_lines[i]))      
    return function_lines

In [71]:
# apply for all IR files in the directory
def setup(directory):
    lines=[]
    functions_lines={} # store each function lines with key as index 0,1,2 .. representing the position of the function in the file
    function_names=[]
    temp_nodes={} # temporary dictionary of nodes for each function
    nodes={} # dictionary of nodes for all functions
    temp_calls=[]
    calls=[] # list of all calls in the CFG tuples (function_name, source_node_id)
    temp_edges=[]
    edges=[]
    functions_entry={} # dict to access the entry node of each function easily by its name
    for filename in os.listdir(directory):
        # print(filename)
        lines.clear()
        f = os.path.join(directory, filename)
        # checking if it is a file
        if os.path.isfile(f):
           #Dict contains each function lines with key as index 0,1,2 .. representing its position in the file
            functions_lines, function_names= extract_functions(directory+'/'+filename) # print file name
            functions_lines=handle_switch(functions_lines)
            print("*****function names ************")
            print(len(functions_lines))
            print(len(function_names))
            # for name in function_names:
            #     print(name)
            print("||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||")
            for name in enumerate( function_names):
                if name[1] =="@_gnu_exception_handler":
                    for line in functions_lines[name[0]]:
                        print(line)
                        print("*********khokha********")

          
            for key in functions_lines:
                # print(key)
                # print("after key")
                # print(len(functions_lines))
                # print("after len lines")
                # print(function_names)
                
                # print("*****************")
                temp_nodes, temp_calls,function_entry= construct_nodes(functions_lines[key], key , function_names[key])
                functions_entry[function_entry[0]]=function_entry[1]
                nodes.update(temp_nodes)
                calls+=temp_calls
         
            create_json(nodes,filename)

            # print("******to2 to2***********")
            # print(len(nodes))
            # for id in nodes.keys():
            #     print(id)
            print("%%%%%%%%%%%%too%%%%%")
            for ir in nodes["140002000"].instructions:
                print(ir)
                print("*********toto********")
            print(nodes["140001000"].instructions[-1])
            
         
            # print(" calls")
            # print(len(calls)) #  
            # # print keys and values of calls dictionary
            # for call in calls:
            #     print(call[0],"  ", call[1], "  ", call[2])

            edges = construct_edges(nodes,functions_entry,calls)
           
            #print(len(functions_lines))
            # # print(functions_lines[5])
            call_edges, nodes = connect_functions(nodes , calls,functions_entry)
            edges.extend(call_edges)
            print("%%%%%%%%%%%%%%%%%%%")
            print(" edges")
            # print(len(edges)) # 10 in test3.exe
            for edge in call_edges:
                print("source: ", edge.source_id)
                print("target: ", edge.target_id)
                print("-------")
            
           
            # print("%%%%%%%%%%%%%%%%%%%")
            # print(" edges")
            # print(len(call_edges))
            # for edge in call_edges:
            #     print("source: ", edge.source_id)
            #     print("target: ", edge.target_id)
            #     print("-------")   
            # create_dataFrame(edges,filename) 
            # print(len(function_names))
            # for f in function_names:
            #     print(f)
            # for call in calls:
            #     print(call)
            #     print(nodes[call[1]].label)
            # for ID in nodes.keys():
            #     print(nodes[ID].instructions[-1])
            # print("edges")
            # print(len(edges))
            # for edge in edges:
            #     print("source: ", edge.source_id)
            #     print("target: ", edge.target_id)
            #     print("-------")
            

In [72]:
setup("IRs")


tooooll1
2
7
7
tooooll2
2
7
7
switch found
2
switch found
54
switch found
92
tooooll1
54
5
5
tooooll2
54
5
5
*****function names ************
85
85
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
define i32 @_gnu_exception_handler(i64 %exception_data) local_unnamed_addr {

*********khokha********
dec_label_pc_140001ed0:

*********khokha********
  %merge.reg2mem = alloca i32, !insn.addr !427

*********khokha********
  %0 = call i64 @__decompiler_undefined_function_0()

*********khokha********
  %1 = trunc i64 %0 to i32

*********khokha********
  %2 = and i32 %1, 553648127, !insn.addr !428

*********khokha********
  %3 = icmp eq i32 %2, 541541187, !insn.addr !429

*********khokha********
  br i1 %3, label %dec_label_pc_140001f90, label %dec_label_pc_140001ef1, !insn.addr !430

*********khokha********


*********khokha********
dec_label_pc_140001ef1:                           ; preds = %dec_label_pc_140001f90, %dec_label_pc_140001ed0

*********khokha********
  %4 = icmp